위에서부터 순서대로 실행하면 되느니라

In [ ]:
# Drive Mount
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# Module Import
from google.colab import drive
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import csv
import collections

from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense, LSTM, Dropout, Flatten
from tensorflow.keras.layers import GlobalMaxPooling1D, MaxPooling1D, Conv1D, Conv2D, Conv3D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization, Activation, Bidirectional, TimeDistributed
from tensorflow.keras.models import Sequential

from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from keras import regularizers
from keras import backend as K
from keras import callbacks

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_squared_error,roc_auc_score,precision_score
from sklearn.metrics import f1_score, recall_score, precision_score, scorer
import sklearn.utils.multiclass

class Metrics(tf.keras.callbacks.Callback):
    def __init__(self, validation):   
        super(Metrics, self).__init__()
        self.validation = validation    
        print('validation shape', len(self.validation[0]))
        
    def on_train_begin(self, logs={}):        
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []
     
    def on_epoch_end(self, epoch, logs={}):
        val_targ = self.validation[1]   
        val_predict = (np.asarray(self.model.predict(self.validation[0]))).round()

        val_f1 = f1_score(val_targ, val_predict)
        val_recall = recall_score(val_targ, val_predict)         
        val_precision = precision_score(val_targ, val_predict)
        
        self.val_f1s.append(round(val_f1, 6))
        self.val_recalls.append(round(val_recall, 6))
        self.val_precisions.append(round(val_precision, 6))
 
        print(f' — val_f1: {val_f1} — val_precision: {val_precision}, — val_recall: {val_recall}')

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
# morph token data load
data = pd.read_csv('/content/drive/Shared drives/BigData/processed_data/labeled_data.csv',index_col=0)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (89,90,220,268,269,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,5

In [ ]:
# space token data
data = pd.read_csv('/content/drive/Shared drives/BigData/processed_data/space_token_labeled_data.csv',index_col=0)

In [ ]:
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#   1. 데이터 전처리
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# remove nan and to List
data_list = []
for d in data.values.tolist():
    data_list.append(list(str(x) for x in d[1:] if pd.notnull(x)))

In [ ]:
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#   2. 데이터 토큰화
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
vocab_size = 994 # 994인 이유는 빈도수 2이상인 단어가 994개
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(data_list)
sequences = tokenizer.texts_to_sequences(data_list)
max_len = max(map(len, data_list))
smishing_text = pad_sequences(sequences, maxlen=max_len)

In [ ]:
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#   3. 학습용 데이터 분류
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
processed_data = pd.concat([data['smishing'], pd.DataFrame(smishing_text)], axis=1)

smishing_data = processed_data[processed_data['smishing'] == 1]
normal_data = processed_data[processed_data['smishing'] == 0]

num_of_smishing = len(smishing_data)
num_of_normal = len(normal_data)

smishing_train_end = 18000
normal_train_end = 2400

smishing_validation_end = int(num_of_smishing*0.8)
normal_validation_end = int(num_of_normal*0.8)

# train data
train_data = pd.concat([
                        smishing_data[:int(smishing_train_end*0.8)], 
                        normal_data[:int(normal_train_end*0.8)]
                        ]).sample(frac=1).reset_index(drop=True)

# validation data
val_data = pd.concat([
                      smishing_data[int(smishing_train_end*0.8):2400], 
                      normal_data[int(normal_train_end*0.8):20000]
                      ]).sample(frac=1).reset_index(drop=True)

# test data
test_data = pd.concat([
                       smishing_data[2400:], 
                       normal_data[20000:]
                       ]).sample(frac=1).reset_index(drop=True)

x_train = np.array(train_data.iloc[:,1:])
y_train = np.array(train_data['smishing'])

x_val = np.array(val_data.iloc[:,1:])
y_val = np.array(val_data['smishing'])

x_test = np.array(test_data.iloc[:,1:])
y_test = np.array(test_data['smishing'])

print("Normal Data")
print(f'{normal_train_end} : {int(num_of_normal*0.2)} : {int(num_of_normal*0.2)}')
print("Smishing Data")
print(f'{smishing_train_end} : {int(num_of_smishing*0.2)} : {int(num_of_smishing*0.2)}')

Normal Data
2400 : 55448 : 55448
Smishing Data
18000 : 3740 : 3740


In [ ]:
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#   4.1 모델 구축 Embedding - CNN - LSTM
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=max_len))
model.add(Dropout(0.25))
model.add(Conv1D(128,
                 5,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=4))
model.add(Bidirectional(LSTM(32)))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[])

In [ ]:
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#   4.2 모델 구축 Embedding - CNN
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
model = Sequential()
model.add(Embedding(vocab_size, 64, input_length=max_len))
model.add(Dropout(0.25))
model.add(Conv1D(64,
                 5,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=4))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[])

In [ ]:
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#   4.3 모델 구축 Embedding - LSTM
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
model = Sequential()
model.add(Embedding(vocab_size, 64, input_length=max_len))
model.add(Bidirectional(LSTM(32)))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[])

In [ ]:
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#   5. 학습
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', verbose=1, patience=2)
mc = tf.keras.callbacks.ModelCheckpoint('best_model.h5',monitor='val_acc', mode='max', verbose=1, save_best_only=True)

# fit the mode
hist = model.fit(
    x_train, 
    y_train,
    validation_data=(x_val,y_val), 
    epochs=20,
    batch_size=512,
    callbacks=[Metrics(validation=(x_val, y_val))],
    verbose=1
)

y_pred = model.predict(x_test, verbose=1).flatten().round()
print(classification_report(y_test, y_pred, target_names=['class 0', 'class 1']))

In [ ]:
#model.save('/content/drive/Shared drives/BigData/smishing_CNN+BILSM_model.h5')

In [ ]:
# install LightBGM
!git clone https://github.com/Microsoft/LightGBM
!cd LightGBM
!mkdir build
!cmake -DUSE_GPU=1
!make -j$(nproc)
!sudo apt-get -y install python-pip
!sudo -H pip install setuptools pandas numpy scipy scikit-learn -U
%cd /content/LightGBM/python-package
!sudo python setup.py install
!pip3 uninstall scikit-learn
!pip3 install scikit-learn==0.21.3

In [ ]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
param_test ={'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

In [ ]:
n_HP_points_to_test = 100

import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

#n_estimators is set to a "large value". The actual number of trees build will depend on early stopping and 5000 define only the absolute maximum
clf = lgb.LGBMClassifier(max_depth=-1, random_state=314, silent=True, metric='None', n_jobs=4, n_estimators=5000)
gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    n_iter=n_HP_points_to_test,
    scoring='roc_auc',
    cv=3,
    refit=True,
    random_state=314,
    verbose=True)

opt_parameters = {'colsample_bytree': 0.9234, 'min_child_samples': 399, 'min_child_weight': 0.1, 'num_leaves': 13, 'reg_alpha': 2, 'reg_lambda': 5, 'subsample': 0.855}

clf_sw = lgb.LGBMClassifier(**clf.get_params())
#set optimal parameters
clf_sw.set_params(**opt_parameters)

In [ ]:
import lightgbm as lgb
from sklearn.metrics import f1_score

def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True

evals_result = {}

X_train = StandardScaler().fit_transform(x_train)
X_val = StandardScaler().fit_transform(x_val)
X_test = StandardScaler().fit_transform(x_test)

Y_train = y_train
Y_val = y_val
Y_test = y_test

dtrain = lgb.Dataset(X_train, Y_train)
dvalid = lgb.Dataset(X_val, Y_val)

#Specifying the parameter
params={}
params['learning_rate']=0.03
params['boosting_type']='gbdt' #GradientBoostingDecisionTree
params['objective']='binary' #Binary target feature
params['metric']='binary_logloss' #metric for binary classification

print("Train Start...")
#train the model 
clf=lgb.train(
    params, 
    dtrain,
    10, 
    valid_sets = dvalid,
    feval=lgb_f1_score, 
    evals_result=evals_result
) #train the model on 100 epocs

Train Start...


In [ ]:
#prediction on the test set
Y_pred=clf.predict(X_test)

#rounding the values
Y_pred=Y_pred.round(0)

#converting from float to integer
Y_pred=Y_pred.astype(int)

print(classification_report(Y_test, Y_pred, target_names=['class 0', 'class 1']))

In [ ]:
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, precision_score
import numpy as np
import os


class Metrics(tf.keras.callbacks.Callback):
    def __init__(self, valid_data):
        super(Metrics, self).__init__()
        self.validation_data = valid_data

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        val_predict = np.argmax(self.model.predict(self.validation_data[0]), -1)
        val_targ = self.validation_data[1]
        if len(val_targ.shape) == 2 and val_targ.shape[1] != 1:
            val_targ = np.argmax(val_targ, -1)

        _val_f1 = f1_score(val_targ, val_predict, average='macro')
        _val_recall = recall_score(val_targ, val_predict, average='macro')
        _val_precision = precision_score(val_targ, val_predict, average='macro')

        logs['val_f1'] = _val_f1
        logs['val_recall'] = _val_recall
        logs['val_precision'] = _val_precision
        print(" — val_f1: %f — val_precision: %f — val_recall: %f" % (_val_f1, _val_precision, _val_recall))
        return


(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=10000, random_state=32)

# LeNet-5
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(32, 32, 3)),
    tf.keras.layers.Conv2D(6, 5, activation='relu'),
    tf.keras.layers.AveragePooling2D(),
    tf.keras.layers.Conv2D(16, 5, activation='relu'),
    tf.keras.layers.AveragePooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(84, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

if not os.path.exists('./checkpoints'):
    os.makedirs('./checkpoints')

#Save the model according to val
ck_callback = tf.keras.callbacks.ModelCheckpoint('./checkpoints/weights.{epoch:02d}-{val_f1:.4f}.hdf5',
                                                 monitor='val_f1', 
                                                 mode='max', verbose=2,
                                                 save_best_only=True,
                                                 save_weights_only=True)
tb_callback = tf.keras.callbacks.TensorBoard(log_dir='./logs', profile_batch=0)
model.fit(x_train, y_train,
          validation_data=(x_val, y_val),
          epochs=100,
          callbacks=[Metrics(valid_data=(x_val, y_val)),
                     ck_callback,
                     tb_callback])